In [ ]:
!pip install protobuf==3.19.5

In [ ]:
!pip install transformers
!pip install qdrant-client>=1.1.1
!pip install pythainlp

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from torch import Tensor
from pythainlp.tokenize import word_tokenize
from qdrant_client import models

In [4]:
import pandas as pd
df = pd.read_csv("/kaggle/input/ticket/data.csv")
df = df.drop(columns=['Unnamed: 0'])
df['sentence'] = df['clean_subject'] + ' ' + df['clean_description']
df = df[df['ticket_type'] != 'Problem'] #Drop Class Problem
df.dropna(subset=['sentence'], inplace=True)
df['sentence'].loc[0]

'ma     น      น   cid  inextbaac   สาขา หน่วยบุรีรัมย์ เรื่อง ขอกำหนดการปรับปรุงการให้บริการ บริษัท ไอเน็กซ์ บรอดแบนด์ จำกัด ขอจัดส่งจดหมายปรับปรุงการบริการวงจร จำนวน  วงจร วัตถุประ์การดำเนินการ   ปรับปรุงบริการโครงข่าย เนื่องจากจะดำเนินการ replace อุปกรณ์ภายใน node พื้นที่ดำเนินการ   node บุรีรัมย์ site interlink วัน เริ่มดำเนินการ       น  วัน สิ้นสุดดำเนินการ       น  ระยะกระทบ   กระทบบริการไม่เกิน   ชั่วโมง รายละเอียดทั้งหมดตาม เอกสารแนบมา จึงมาโปรดทราบความจำเป็นดังกล่าว และบริษัทฯ ขออภัยในความไม่สะดวกในครั้งนี้ หากท่านมีข้อสงสัยหรือต้องการข้อมูลเพิ่มเติม โปรดติดต่อตามหมายเลขด้านล่าง หรือ reply mail กลับมาที่'

In [5]:
doc = df[['sentence', 'ticket_type']].sample(n=10)
sen_list = doc.to_dict(orient='records')
sen_list[0]

{'sentence': "ปรับปรุงระบบฐานข้อมูล แก้ไขปรับปรุง on  september   at    pm gmt  inet network operation wrote  ผู้ใช้บริการ เนื่องจากทางฝ่ายปฏิบัติการ บริษัท อินเทอร์เน็ตประเทศไทย จำกัด จะดำเนินการปรับปรุงระบบฐานข้อมูล เพื่อความรวดเร็ว และถูกต้องสำหรับการติดต่อผู้ใช้บริการ กับทางฝ่ายปฏิบัติการรวมทั้งการข้อมูล ข่าวสาร สำหรับผู้ใช้บริการให้ถูกต้องมากยิ่งขึ้น ดังนั้นทางฝ่ายฯ รบกวนผู้ใช้บริการอัพเดตฐานข้อมูลในส่วนของผู้ติดต่อประสานงาน และทำการระบุสิทธิ์ที่ได้รับของแต่ละท่าน โดยรายละเอียดสิทธิ์ที่สามารถดำเนินการได้มีรายละเอียดเพิ่มเติมตามตารางในไฟล์แนบ ทั้งนี้หากไม่มีการตอบกลับ ทางฝ่ายฯ จะอ้างอิงตามฐานข้อมูลเดิม หมายเหตุ   หากบริษัทของผู้ใช้บริการ ต้องการระบุสิทธิ์ของเจ้าหน้าที่นอกเหนือจากที่บริษัทฯได้ไว้ กรุณาระบุเพิ่มเติม   หากทางผู้ใช้บริการไม่ได้ทำการระบุสิทธิ์ที่ได้รับ หรือระบุเจ้าหน้าที่ที่ดูแลในแต่ละ service ทางบริษัทฯ ขออนุญาตปรับสิทธิ์เป็น authorized third level และระบุเป็นเจ้าหน้าที่ที่ดูแลทุก service ที่มีการใช้บริการกับบริษัทฯ โดยอัตโนมัติ   การดำเนินการอื่นนอกเหนือจากตารางตามไฟล

In [ ]:
!pip install --upgrade tensorflow pythainlp

In [ ]:
!pip install deepcut

In [8]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Using PyThaiNLP to tokenize the input texts
input_texts = [' '.join(word_tokenize(doc['sentence'], engine="deepcut")) for doc in sen_list]

# Load transformers model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large-instruct')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large-instruct')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

# Get model outputs
outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# Normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)

# Connect to Qdrant
qdrant = QdrantClient(":memory:")

# Create a collection in Qdrant
qdrant.recreate_collection(
    collection_name="test",
    vectors_config=models.VectorParams(
        size=embeddings.size(-1),  # Use the size of the embeddings
        distance=models.Distance.COSINE
    )
)

# Prepare points to insert
points = [
    PointStruct(
        id=i,
        vector=embeddings[i].tolist(),
        payload={"document": sen_list[i]}
    )
    for i in range(len(sen_list))
]

# Insert points into the collection
qdrant.upsert(
    collection_name="test",
    points=points
)

2024-07-05 04:37:33.183247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 04:37:33.280315: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 04:37:33.280390: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Inp

67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

I0000 00:00:1720154261.186822  141374 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


68/68 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 309ms/step
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 593ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 269ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 213ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 315ms/step


config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

/tmp/ipykernel_141304/839133901.py:26: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
# Example query
query = 'รบกวนตรวจสอบ'
query_text = ' '.join(word_tokenize(query, engine="deepcut"))
query_dict = tokenizer(query_text, max_length=512, padding=True, truncation=True, return_tensors='pt')

query_outputs = model(**query_dict)
query_embedding = average_pool(query_outputs.last_hidden_state, query_dict['attention_mask'])

query_embedding = F.normalize(query_embedding, p=2, dim=1)

# Search in Qdrant
search_result = qdrant.search(
    collection_name="test",
    query_vector=query_embedding[0].tolist(),
    limit=10  # Number of nearest neighbors to return
)

# Output search results
for i, hit in enumerate(search_result):
    print(f"Result {i+1}:")
    print("Score:", hit.score)
    print("Payload:", hit.payload)
    print(f"--------------------")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Result 1:
Score: 0.8621081098821028
Payload: {'document': {'sentence': 'ตรวจสอบพบอุปกรณ์ที่ส่ง log มายังศูนย์ soc หยุดส่ง log source stop noc ครับ subject   inethc  hospital somdej  inc   high  alert  monitor log source stop detail  กรุณาส่งรายละเอียดด้านล่างให้กับ team  inet co csirt เพื่อตรวจสอบและแก้ไขปัญหาต่อไป                                                                      รบกวนประสานงานลูกค้า  inet enduser โรงพยาบาลสมเด็จพระบรมราชเทวี ณ ศรีราชา  ให้ตรวจสอบเครื่อง ip ดังกล่าว เนื่องจากทางทีมได้รับ alert จากระบบว่า ip ดังกล่าวหยุดส่ง log กรุณาส่งรายละเอียดด้านล่างให้กับหน่วยงานที่เกี่ยวข้องเพื่อตรวจสอบและแก้ไขปัญหาต่อไป                                                                      หัวข้อ  ตรวจพบอุปกรณ์ของ  inet enduser โรงพยาบาลสมเด็จพระบรมราชเทวี ณ ศรีราชา  หยุดส่ง log โดยมีรายละเอียดดังนี้ พบอุปกรณ์ที่ส่ง log มายังศูนย์ soc หยุดส่ง log inethc somdej y f  sw aruba     inethc somdej blay hp f hp     อุปกรณ์ หรือระบบที่พบปัญหา อุปกรณ